In [1]:
import numpy as np
p = 3000
n = int(p/2)
niter = 11
comp_time_fista = np.zeros((niter,6))

In [2]:
from ctypes import *
import numpy as np
import time

np.random.seed(2022)
X = np.random.randn(n,p).astype(np.float64)
tr_beta = np.zeros(p).astype(np.float64)
tr_beta[:int(0.05*p)] = 1.0
y = np.dot(X, tr_beta) + np.random.randn(n).astype(np.float64)

libc_FISTA_dll = CDLL("/home/dyu/Dropbox/tf-notebooks/cuda_comp/fista_example/using_dll/fista_dll_double.so")
FISTA_Kernel_D = libc_FISTA_dll.FISTA_D

FISTA_Kernel_D.restype = None
FISTA_Kernel_D.argtypes = (POINTER(c_double), POINTER(c_double), POINTER(c_double), 
                         POINTER(c_double), POINTER(c_double), POINTER(c_double), POINTER(c_double), POINTER(c_double),
                         POINTER(c_int), POINTER(c_int),POINTER(c_int),POINTER(c_int))



for i in np.arange(niter):
    beta = np.zeros(p).astype(np.float64)
    beta_cr = beta.ctypes.data_as(POINTER(c_double))
    X_cr =  X.ctypes.data_as(POINTER(c_double)) 
    y_cr = y.ctypes.data_as(POINTER(c_double))


    loss = np.zeros(5000).astype(np.float64)## max_iter size만큼
    loss_c = loss.ctypes.data_as(POINTER(c_double)) 
    lam =  np.sqrt(2 * np.log(p) / n)
    lam_c = c_double(lam)
    L_c = c_double(10)
    eta_c = c_double(2.0)
    tol_c = c_double(1e-03)

    iter_c = c_int(5000)
    n_c = c_int(n)
    p_c = c_int(p)
    step_c = c_int(0)

    t1 = time.time()
    FISTA_Kernel_D(beta_cr, X_cr, y_cr, lam_c, L_c, eta_c, tol_c, loss_c, iter_c, n_c, p_c, step_c)
    t2 = time.time()
    comp_time_fista[i,0] = t2-t1

    print('\n ',i+1,'-th iteration, Collapsed Time: ', comp_time_fista[i,0])
    




  1 -th iteration, Collapsed Time:  2.9628093242645264

  2 -th iteration, Collapsed Time:  2.808971405029297

  3 -th iteration, Collapsed Time:  2.8115196228027344

  4 -th iteration, Collapsed Time:  2.8106613159179688

  5 -th iteration, Collapsed Time:  2.825444459915161

  6 -th iteration, Collapsed Time:  2.815269708633423

  7 -th iteration, Collapsed Time:  2.8156471252441406

  8 -th iteration, Collapsed Time:  2.8087282180786133

  9 -th iteration, Collapsed Time:  2.8160183429718018

  10 -th iteration, Collapsed Time:  2.8074920177459717

  11 -th iteration, Collapsed Time:  2.818000078201294


In [3]:
step_c

c_int(1764)

In [4]:
beta_cr[:5]

[0.9540745654624302,
 0.9769645522846867,
 0.9527462243312763,
 0.9482421034939036,
 1.0477021786729235]

In [5]:
from ctypes import *
import numpy as np
import time


np.random.seed(2022)
X = np.random.randn(n,p).astype(np.float64)
tr_beta = np.zeros(p).astype(np.float64)
tr_beta[:int(0.05*p)] = 1.0
y = np.dot(X, tr_beta) + np.random.randn(n).astype(np.float64)

libc_FISTA_blas = CDLL("/home/dyu/Dropbox/tf-notebooks/cuda_comp/fista_example/using_dll/fista_cublas_double.so")
FISTA_blas = libc_FISTA_blas.FISTA

FISTA_blas.restype = None
FISTA_blas.argtypes = (POINTER(c_double), POINTER(c_double), POINTER(c_double), 
                         POINTER(c_double), POINTER(c_double), POINTER(c_double), POINTER(c_double), POINTER(c_double),
                         POINTER(c_int), POINTER(c_int),POINTER(c_int),POINTER(c_int))

for i in np.arange(niter):
    
    beta = np.zeros(p).astype(np.float64)
    beta_cc = beta.ctypes.data_as(POINTER(c_double))
    X_c =  X.ravel(order = "F")              ## n*p 에 () 반드시 필요, c_double * dim 형태만 지원 
    X_cc = X_c.ctypes.data_as(POINTER(c_double))
    y_cc = y.ctypes.data_as(POINTER(c_double))

    loss = np.zeros(5000).astype(np.float64)## max_iter size만큼
    loss_c = loss.ctypes.data_as(POINTER(c_double)) 
    lam =  np.sqrt(2 * np.log(p) / n).astype(np.float64)
    lam_c = c_double(lam)
    L_c = c_double(10)
    eta_c = c_double(2.0)
    tol_c = c_double(1e-03)

    iter_c = c_int(5000)
    n_c = c_int(n)
    p_c = c_int(p)
    step_c = c_int(0)

    t1 = time.time()
    FISTA_blas(beta_cc, X_cc, y_cc, lam_c, L_c, eta_c, tol_c, loss_c, iter_c, n_c, p_c, step_c)
    t2 = time.time()
    comp_time_fista[i,1] = t2-t1

    print('\n ',i+1,'-th iteration, Collapsed Time: ', comp_time_fista[i,1])
    



  1 -th iteration, Collapsed Time:  1.1256682872772217

  2 -th iteration, Collapsed Time:  0.8197908401489258

  3 -th iteration, Collapsed Time:  0.8013098239898682

  4 -th iteration, Collapsed Time:  0.7864444255828857

  5 -th iteration, Collapsed Time:  0.8005800247192383

  6 -th iteration, Collapsed Time:  0.810218095779419

  7 -th iteration, Collapsed Time:  0.7859656810760498

  8 -th iteration, Collapsed Time:  0.7888820171356201

  9 -th iteration, Collapsed Time:  0.7885239124298096

  10 -th iteration, Collapsed Time:  0.8153805732727051

  11 -th iteration, Collapsed Time:  0.7951254844665527


In [6]:
step_c

c_int(1741)

In [7]:
beta_cc[:5]

[0.9553229786941347,
 0.9786208047010384,
 0.9601231139438773,
 0.9494173752605825,
 1.0526504581998382]

In [8]:
import pycuda.driver as cuda
import pycuda.autoinit
from pycuda.compiler import SourceModule
from pycuda import gpuarray, tools, cumath
import numpy as np
import time
import math

with open('/home/dyu/Dropbox/tf-notebooks/cuda_comp/fista_example/pycuda/FISTA_kernel.cu') as f:
    FISTA_kernel = f.read()
    
FISTA_module = SourceModule(FISTA_kernel)

Dgemv = FISTA_module.get_function("Dgemv")
soft_thresh_D = FISTA_module.get_function("soft_thresh_D")

np.random.seed(2022)
X = np.random.randn(n,p).astype(np.float64)
tr_beta = np.zeros(p).astype(np.float64)
tr_beta[:int(0.05*p)] = 1.0
y = np.dot(X, tr_beta) + np.random.randn(n).astype(np.float64)


def FISTA_D(beta, X, y, lam, L, eta, tol = 1e-04, max_iter = 5000):
    n = np.int32(X.shape[0])
    p = np.int32(X.shape[1])
    t = np.float64(1.0)
    One = np.float64(1.0)
    MOne = np.float64(-1.0)
    IntOne = np.int32(1)
    IntZero = np.int32(0)
    
    crit = np.zeros(max_iter, dtype=np.float64)
    temp = np.zeros(p, dtype=np.float64)
    d_beta_p = gpuarray.to_gpu(temp)
    d_beta_prev = gpuarray.to_gpu(temp)
    d_X = gpuarray.to_gpu(X)
    d_y = gpuarray.to_gpu(y)
    d_ymXbp = gpuarray.empty(n, dtype = np.float64)
    d_beta = gpuarray.empty(p, dtype = np.float64)
  
    TPB = (32, 1, 1)
    bpg_p = math.ceil(np.float64(p)/TPB[0])
    bpg_n = math.ceil(np.float64(n)/TPB[0])
    BPG_p = (bpg_p, 1, 1)
    BPG_n = (bpg_n, 1, 1)
  
    L_prev = L
    for k in range(max_iter):
        d_ymXbp = d_y.copy()
        Dgemv(MOne, d_X, d_beta_p, 
              d_ymXbp, n, p, IntZero,
              grid = BPG_n, block = TPB)
        h_rbp = gpuarray.dot(d_ymXbp, d_ymXbp)
        d_XTrbp = gpuarray.zeros(p, np.float64)
        Dgemv(One, d_X, d_ymXbp, d_XTrbp, n, p,
              IntOne, grid = BPG_p, block = TPB)
        
        i_k = -1
        cond = True
        while cond:
            i_k += 1
            eta_ik = eta ** i_k
            L_cur = L_prev * eta_ik
            alpha = np.float64(1.0/L_cur)  
            d_bstar = d_beta_p + alpha*d_XTrbp
            alpha = np.float64(lam/L_cur)
            soft_thresh_D(d_bstar, alpha, d_beta, p,
                         grid = BPG_p, block = TPB)
            d_diff_beta = d_beta - d_beta_p
            h_RHS_1st = gpuarray.dot(d_diff_beta,
                                     d_diff_beta)
            h_RHS_2nd = gpuarray.dot(d_diff_beta,
                                     d_XTrbp)
              
            RHS1 = L_cur * h_RHS_1st.get()
            RHS2 = np.float32(2.0)*h_RHS_2nd.get()
            RHS =  RHS1 - RHS2 
                              
            d_ymXb = d_y.copy()
            Dgemv(MOne, d_X, d_beta, d_ymXb, n, p, 
                  IntZero, grid = BPG_n, block = TPB)
            d_ymXb2 = gpuarray.dot(d_ymXb, d_ymXb)
            LHS = d_ymXb2.get() - h_rbp.get()
            cond = (LHS > RHS)
          
        L_prev = L_cur
        tnext = np.float64((1.0+np.sqrt(1+4*t*t))/2)
        d_diff_beta = d_beta - d_beta_prev
        alpha = np.float64((t - 1.0)/tnext)
        d_beta_p = d_beta + alpha * d_diff_beta
        d_diff_b_sq = gpuarray.dot(d_diff_beta,
                                   d_diff_beta)
        crit[k] = np.sqrt(d_diff_b_sq.get())
          
        if crit[k] < tol:
            break
          
        t = tnext
        d_beta_prev = d_beta.copy()
          
    return d_beta.get(), crit, k

for i in np.arange(niter):

    beta = np.zeros(p, dtype = np.float64)
    lam = np.sqrt(2*np.log(p)/n).astype(np.float64)
    L = np.float64(10)
    eta = np.float64(2)
    
    t1 = time.time()
    out, crit, step = FISTA_D(beta, X, y, lam, L, eta, tol = 1e-3, max_iter = 5000)
    t2 = time.time()
    comp_time_fista[i,2] = t2-t1
    
    print('\n ',i+1,'-th iteration, Collapsed Time: ', comp_time_fista[i,2])




  1 -th iteration, Collapsed Time:  5.43377685546875

  2 -th iteration, Collapsed Time:  4.752575397491455

  3 -th iteration, Collapsed Time:  4.741160154342651

  4 -th iteration, Collapsed Time:  4.670172929763794

  5 -th iteration, Collapsed Time:  4.850614070892334

  6 -th iteration, Collapsed Time:  4.861163854598999

  7 -th iteration, Collapsed Time:  4.772057771682739

  8 -th iteration, Collapsed Time:  4.741984128952026

  9 -th iteration, Collapsed Time:  4.869396686553955

  10 -th iteration, Collapsed Time:  4.714514255523682

  11 -th iteration, Collapsed Time:  4.935123682022095


In [9]:
step

1755

In [10]:
out[:5]

array([0.95591871, 0.97639646, 0.95139887, 0.95003793, 1.05218448])

In [11]:
import numpy as np
from numba import cuda
import math
import time

@cuda.jit
def gemv(alpha, A, x, y, tran):
    Row = cuda.grid(1)
    n = A.shape[0]
    p = A.shape[1]
    pvalue = 0.
    if tran:
        if Row < p:
            for i in range(n):
                pvalue += A[i, Row] * x[i]
            y[Row] += alpha * pvalue
    else:
        if Row < n:
            for j in range(p):
                pvalue += A[Row, j] * x[j]
            y[Row] += alpha * pvalue

@cuda.jit
def soft_thr(x, alpha, S):
    idx = cuda.grid(1)
    if idx < x.shape[0]:
        thr = math.fabs(x[idx]) - alpha
        if thr > 0:
            S[idx] = math.copysign(thr, x[idx])
        else:
            S[idx] = 0

@cuda.jit
def axpy(alpha, x, y):
    idx = cuda.grid(1)
    if idx < x.shape[0]:
        y[idx] += alpha * x[idx]

        
@cuda.jit
def vec_prod(x, y, res):
    idx = cuda.grid(1)
    if idx < x.shape[0]:
        res[idx] = x[idx] * y[idx]

@cuda.reduce
def reduce(x, y):
    return x+y

def FISTA(beta, X, y, lamb, L = 10, eta = 1.2, tol = 1e-08, 
          max_iter = 5000, dtype = np.float32):
    n = X.shape[0]
    p = X.shape[1]
    t = dtype(1.0)
    crit = np.zeros(max_iter, dtype = dtype)
    d_bstar = np.zeros(p, dtype = dtype)
    d_beta = np.zeros(p, dtype = dtype)
    d_diff_beta2 = np.zeros(p, dtype = dtype)
    d_XTrbpd = np.zeros(p, dtype = dtype)
    d_beta_prev = np.zeros(p,dtype = dtype)
    d_beta_p = np.zeros(p, dtype = dtype)
    d_ymXbp2 = np.zeros(n, dtype = dtype)
    d_ymXb2= np.zeros(n, dtype = dtype)
    L_prev = L
    TPB = (32,1)
    BPG_p = (math.ceil(p / 32) , 1)
    BPG_n = (math.ceil(n / 32) , 1)
    
    for k in range(max_iter):
        d_ymXbp = y.copy()
        gemv[BPG_n, TPB](-1.0, X, d_beta_p, d_ymXbp, False)
        vec_prod[BPG_n, TPB](d_ymXbp, d_ymXbp, d_ymXbp2)
        h_rbp = reduce(d_ymXbp2)
        d_XTrbp = np.zeros(p, dtype = dtype)
        gemv[BPG_p, TPB](1.0, X, d_ymXbp, d_XTrbp, True)
        
        i_k = -1
        cond = True
        while cond:
            i_k += 1
            L_cur = L_prev * (eta ** i_k)
            d_bstar = d_beta_p.copy()
            axpy[BPG_p, TPB]((1.0 / L_cur), d_XTrbp, d_bstar)
            soft_thr[BPG_p, TPB](d_bstar, lamb / L_cur, d_beta)
            d_diff_beta = d_beta.copy()
            axpy[BPG_p, TPB](-1.0, d_beta_p, d_diff_beta)
            vec_prod[BPG_p, TPB](d_diff_beta, d_diff_beta, d_diff_beta2)
            vec_prod[BPG_p, TPB](d_XTrbp, d_diff_beta, d_XTrbpd)
            RHS = L_cur * reduce(d_diff_beta2) - 2.0 * reduce(d_XTrbpd)
            d_ymXb = y.copy()
            gemv[BPG_n, TPB](-1.0, X, d_beta, d_ymXb, False)
            vec_prod[BPG_n, TPB](d_ymXb, d_ymXb, d_ymXb2)
            h_rb = reduce(d_ymXb2)
            LHS =  h_rb - h_rbp
            cond = (LHS > RHS)
        
        L_prev = L_cur
        tnext = (1.0 + np.sqrt(1 + 4 * t**2) ) / 2.0
        d_diff_beta = d_beta.copy()
        axpy[BPG_p, TPB](-1.0, d_beta_prev, d_diff_beta)
        t1 = (t - 1.0) / tnext
        d_beta_p = d_beta.copy()
        axpy[BPG_p, TPB](t1, d_diff_beta, d_beta_p)
        vec_prod[BPG_p, TPB](d_diff_beta, d_diff_beta, d_diff_beta2)
        crit[k] = np.sqrt(reduce(d_diff_beta2))
        if crit[k] < tol:
            break
        t = tnext
        d_beta_prev = d_beta.copy()
    return d_beta, crit, k

np.random.seed(2022)
X = np.random.randn(n,p).astype(np.float64)
tr_beta = np.zeros(p).astype(np.float64)
tr_beta[:int(0.05*p)] = 1.0
y = np.dot(X, tr_beta) + np.random.randn(n).astype(np.float64)

cuda.select_device(0)
for i in np.arange(niter):

    beta = np.zeros(p, dtype = np.float64)
    lam = np.sqrt(2*np.log(p)/n).astype(np.float64)
    L = np.float64(10)
    eta = np.float64(2)
    
    t1 = time.time()
    out, crit, step = FISTA(beta, X, y, lam, L, eta, tol = 1e-03, max_iter = 5000, dtype=np.float64)
    t2 = time.time()
    comp_time_fista[i,3] = t2-t1
    
    print('\n ',i+1,'-th iteration, Collapsed Time: ', comp_time_fista[i,3])
    



  1 -th iteration, Collapsed Time:  113.32188868522644

  2 -th iteration, Collapsed Time:  113.04587888717651

  3 -th iteration, Collapsed Time:  113.21139931678772

  4 -th iteration, Collapsed Time:  111.98239946365356

  5 -th iteration, Collapsed Time:  112.53168272972107

  6 -th iteration, Collapsed Time:  113.60604357719421

  7 -th iteration, Collapsed Time:  113.19279909133911

  8 -th iteration, Collapsed Time:  112.07095694541931

  9 -th iteration, Collapsed Time:  112.46536064147949

  10 -th iteration, Collapsed Time:  111.88113498687744

  11 -th iteration, Collapsed Time:  112.5671615600586


In [12]:
step

1758

In [13]:
out[:5]

array([0.95583119, 0.97726276, 0.95160533, 0.9520176 , 1.05104607])

In [14]:
import torch
import numpy as np
import time
import gc

gc.collect()
torch.cuda.empty_cache()
torch.cuda.init()
torch.backends.cuda.matmul.allow_tf32 = False

def soft_thr(x, alpha):
    n = x.shape[0]
    S = torch.maximum(torch.abs(x)-alpha, 
        torch.zeros(n,device='cuda:0'))*torch.sign(x)
    return S

def FISTA(beta, X, y, lamb, L, eta , tol = 1e-04, max_iter = 5000, dtype = torch.float32):
    if(dtype == torch.float32):
        torch.set_default_tensor_type(torch.FloatTensor)
    else:
        torch.set_default_tensor_type(torch.DoubleTensor)
    device = torch.device("cuda:0")
    n = X.shape[0]
    p = X.shape[1]
    dbeta = torch.Tensor(beta).to(device)
    dX = torch.Tensor(X).to(device)
    dy = torch.Tensor(y).to(device)
    t = torch.ones(1,dtype=dtype, device=device)
    crit = np.zeros(max_iter)
    dbeta_p = torch.Tensor(beta).to(device)
    dbeta_prev = torch.Tensor(beta).to(device)
    L_prev = L
    for k in range(max_iter):
        dymXbp = dy-torch.matmul(dX,dbeta_p)
        drbp = torch.dot(dymXbp, dymXbp)
        dXTrbp = torch.matmul(torch.t(dX),dymXbp)

        i_k = -1
        cond = True
        while cond:
            i_k += 1
            L_cur = L_prev*(eta**i_k)
            dbstar = dbeta_p + dXTrbp/L_cur
            dbeta = soft_thr(dbstar, lamb/L_cur)
            diff_beta = dbeta - dbeta_p
            RHS_1st = torch.dot(diff_beta,diff_beta)
            RHS_2nd = torch.dot(diff_beta,dXTrbp)
            RHS = L_cur*RHS_1st-2.0*RHS_2nd
            dymXb = dy-torch.matmul(dX,dbeta)
            LHS = torch.dot(dymXb,dymXb)-drbp
            cond = (LHS>RHS)
        L_prev = L_cur
        tnext = (1.0+torch.sqrt(1+4*t**2))/2.0
        diff_beta = dbeta-dbeta_prev
        t1 = (t-1.0)/tnext
        dbeta_p = dbeta+t1*diff_beta
        crit[k] = torch.linalg.norm(diff_beta).to('cpu')
        if crit[k] < tol:
            break
        t = tnext
        dbeta_prev = dbeta
    out = dbeta.to('cpu')
    return out.numpy(), crit, k

np.random.seed(2022)
X = np.random.randn(n,p).astype(np.float64)
tr_beta = np.zeros(p).astype(np.float64)
tr_beta[:int(0.05*p)] = 1.0
y = np.dot(X, tr_beta) + np.random.randn(n).astype(np.float64)


for i in np.arange(niter):

    beta = np.zeros(p, dtype = np.float64)
    lam = np.sqrt(2*np.log(p)/n).astype(np.float64)
    L = np.float64(10)
    eta = np.float64(2)
    
    t1 = time.time()
    out,crit,step = FISTA(beta, X, y, lam, L, eta, tol = 1e-03, max_iter = 5000, dtype = torch.float64)
    t2 = time.time()
    comp_time_fista[i,5] = t2-t1
    
    print('\n ',i+1,'-th iteration, Collapsed Time: ', comp_time_fista[i,5])
   




  1 -th iteration, Collapsed Time:  3.04213809967041

  2 -th iteration, Collapsed Time:  1.127195119857788

  3 -th iteration, Collapsed Time:  1.107191562652588

  4 -th iteration, Collapsed Time:  1.2018523216247559

  5 -th iteration, Collapsed Time:  1.0588312149047852

  6 -th iteration, Collapsed Time:  1.0352919101715088

  7 -th iteration, Collapsed Time:  1.051959753036499

  8 -th iteration, Collapsed Time:  1.2226064205169678

  9 -th iteration, Collapsed Time:  1.0293498039245605

  10 -th iteration, Collapsed Time:  1.0370068550109863

  11 -th iteration, Collapsed Time:  1.120659351348877


In [15]:
step

1759

In [16]:
out[:5]

array([0.95254066, 0.9785757 , 0.95588044, 0.95169777, 1.05268715])

In [17]:
crit[:9]

array([3.14292947, 1.54338943, 1.21923763, 0.96647247, 0.78125418,
       0.64401647, 0.53842078, 0.45384516, 0.38440976])

In [18]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [19]:
import tensorflow as tf
import numpy as np
import time

@tf.function
def soft_thr(x,alpha):
    n = x.shape[0]
    S = tf.math.maximum(tf.abs(x)-alpha,0)*tf.math.sign(x)
    return S

def FISTA(beta, X, y, lamb, L, eta, tol = 1e-08, max_iter = 5000, dtype = tf.float32):
    n = X.shape[0]
    p = X.shape[1]
    crit = np.zeros(max_iter)
  
    with tf.device("/GPU:0"):
        dbeta = tf.constant(beta)
        dX = tf.constant(X)
        dy = tf.constant(y)
        t = tf.constant(1.0, dtype=dtype)
        dbeta_p = tf.constant(beta)
        dbeta_prev = tf.constant(beta)
        L_prev = tf.constant(L,dtype=dtype)

        for k in range(max_iter):
            dymXbp = dy-tf.linalg.matvec(dX,dbeta_p)
            drbp = tf.tensordot(dymXbp,dymXbp,axes=1)
            dXTrbp = tf.linalg.matvec(dX,dymXbp, 
                                    transpose_a=True)

            i_k = -1
            cond = True
            while cond:
                i_k += 1
                L_cur = L_prev*(eta**i_k)
                dbstar = dbeta_p + dXTrbp/L_cur
                dbeta = soft_thr(dbstar, lamb/L_cur)
                diff_beta = dbeta - dbeta_p
                RHS_1st = tf.tensordot(diff_beta, 
                                       diff_beta,axes=1)
                RHS_2nd = tf.tensordot(diff_beta, dXTrbp,
                                       axes=1)
                RHS = L_cur * RHS_1st.numpy() - 2.0 * RHS_2nd.numpy()
                dymXb = dy - tf.linalg.matvec(dX,dbeta)
                LHS = (tf.tensordot(dymXb,dymXb,
                       axes=1)).numpy()-drbp.numpy()
                cond = (LHS>RHS)
            L_prev = L_cur
            tnext =  (1.0+tf.sqrt(1+4*t**2))/2.0
            diff_beta = dbeta-dbeta_prev
            t1 = (t - 1.0)/tnext
            dbeta_p = dbeta+t1*diff_beta
            crit[k] = tf.norm(diff_beta) 
            if crit[k] < tol:
                break
            t = tnext
            dbeta_prev = dbeta
    return dbeta.numpy(), crit, k

np.random.seed(2022)
X = np.random.randn(n,p).astype(np.float64)
tr_beta = np.zeros(p).astype(np.float64)
tr_beta[:int(0.05*p)] = 1.0
y = np.dot(X, tr_beta) + np.random.randn(n).astype(np.float64)

for i in np.arange(niter):

    beta = np.zeros(p, dtype = np.float64)
    lam = np.sqrt(2*np.log(p)/n).astype(np.float64)
    L = np.float64(10)
    eta = np.float64(2)
    
    t1 = time.time()
    out, crit, step = FISTA(beta, X, y, lam, L, eta, tol = 1e-03, max_iter = 5000, dtype=tf.float64)
    t2 = time.time()
    comp_time_fista[i,4] = t2-t1
    
    print('\n ',i+1,'-th iteration, Collapsed Time: ', comp_time_fista[i,4])
   



  1 -th iteration, Collapsed Time:  8.79351019859314

  2 -th iteration, Collapsed Time:  6.609880447387695

  3 -th iteration, Collapsed Time:  6.880760431289673

  4 -th iteration, Collapsed Time:  6.944295883178711

  5 -th iteration, Collapsed Time:  6.326539516448975

  6 -th iteration, Collapsed Time:  6.387064456939697

  7 -th iteration, Collapsed Time:  6.358918190002441

  8 -th iteration, Collapsed Time:  6.475696563720703

  9 -th iteration, Collapsed Time:  6.693721055984497

  10 -th iteration, Collapsed Time:  8.371906518936157

  11 -th iteration, Collapsed Time:  7.335567951202393


In [20]:
step

1764

In [21]:
out[:5]

array([0.96021451, 0.97877206, 0.95417989, 0.95321325, 1.05189365])

In [22]:
with open('./fista_comp_time_n{}_p{}_double.txt'.format(n,p), 'w') as f:
    print(('%s\t '*6 % ('DLL-K','DLL-cuBLAS','PyCUDA','Numba','TF','PyTorch')), end='\n', file=f)
    for i in range(niter):
        print(('%.4f\t '*6 % tuple(comp_time_fista[i])), end='\n', file=f)
